# Searching A Single Document - Rijkswaterstaat (RWS)

A test on how to load a PDF document and search it using the Perry AI Powered Document Analysis tool.

## Constructing The File Nodes

In [1]:
# Check if doc_data/RWS exists, give warning if not
import pathlib

def get_data_path():
    """Returns the path to the RWS data directory."""
    return pathlib.Path.cwd() / "doc_data" / "RWS"

rws_path = get_data_path()
if not rws_path.exists():
    print("Warning: RWS data not found. Please create the directory and fill it with document data first.")

if not any(rws_path.iterdir()):
    print("Warning: RWS data not found. Please fill the directory with document data first.")

In [2]:
import os
import openai
from dotenv import load_dotenv
from llama_index import VectorStoreIndex, StorageContext, ServiceContext, load_index_from_storage
from llama_index.readers.base import BaseReader
from llama_index.schema import Document

def load_openai_api_key():
    """Load OpenAI API key from environment variable."""
    load_dotenv()
    openai.api_key = os.environ["OPENAI_API_KEY"]

def get_rws_file_paths(data_dir: pathlib.Path) -> list[pathlib.Path]:
    """Get file paths of RWS documents."""
    return [f for f in data_dir.iterdir() if f.is_file()]

def load_rws_documents(data_dir: pathlib.Path, loader: BaseReader) -> dict[str, list[Document]]:
    """Load RWS documents."""
    doc_set = {}
    for doc_path in get_rws_file_paths(data_dir):
        docs = loader.load_data(file=doc_path, split_documents=False)
        doc_set[doc_path.stem] = docs
    return doc_set
    
def create_vector_indexes(doc_set: dict[str, list[Document]]) -> dict[str, list[VectorStoreIndex]]:
    """Create and store vector indexes."""
    service_context = ServiceContext.from_defaults(chunk_size=512)
    vector_indexes = {}
    for doc_name, docs in doc_set.items():
        storage_context = StorageContext.from_defaults()
        vector_index = VectorStoreIndex.from_documents(
            docs,
            service_context=service_context,
            storage_context=storage_context
        )
        vector_indexes[doc_name] = vector_index
    return vector_indexes

def save_vector_indexes(vector_indexes: dict[str, list[VectorStoreIndex]]) -> None:
    """Save vector indexes."""
    for doc_name, vector_index in vector_indexes.items():
        vector_index.storage_context.persist(persist_dir=f'./storage/rws/{doc_name}')

def load_vector_indexes() -> dict[str, list[VectorStoreIndex]]:
    """Load vector indexes."""
    vector_indexes = {}
    for doc_name in get_rws_file_paths(rws_path):
        storage_context = StorageContext.from_defaults(persist_dir=f'./storage/rws/{doc_name.stem}')
        vector_index = load_index_from_storage(storage_context=storage_context)
        vector_indexes[doc_name.stem] = vector_index
    return vector_indexes


In [6]:
load_openai_api_key()

In [5]:
# Load in docs
from llama_index import download_loader
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)
loader = UnstructuredReader()
docs = load_rws_documents(rws_path, loader)

[nltk_data] Downloading package punkt to /home/bluegnome/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/bluegnome/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
vector_indexes_new = create_vector_indexes(docs)
save_vector_indexes(vector_indexes_new)

In [3]:
vector_indexes = load_vector_indexes()

In [11]:
vector_indexes_new.keys()

dict_keys(['Aanbestedingsleidraad_BOC_ZD'])

In [12]:
vector_indexes.keys()

dict_keys(['Aanbestedingsleidraad_BOC_ZD'])

# Create Langchain Tooling

Create langchain tooling from the loaded indexes.

In [4]:
from langchain.chains.conversation.memory import ConversationBufferMemory

from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig
from llama_index.bridge.langchain import AgentExecutor

from langchain.chat_models import ChatOpenAI


def create_vector_index_tool_configs(vector_indexes: dict[str, list[VectorStoreIndex]]) -> dict[str, IndexToolConfig]:
    """Create vector index tool configs."""
    index_tool_configs = {}
    for doc_name, vector_index in vector_indexes.items():
        query_engine = vector_index.as_query_engine(
            similarity_top_k=3,
        )
        index_tool_configs[doc_name] = IndexToolConfig(
            query_engine=query_engine,
            name=f"Vector Index {doc_name}",
            description=f"Aanbestedingsleidraad voor rijkswaterstaat opdrachten",
            tool_kwargs={"return_direct": True}
        )
    return index_tool_configs

def create_langchain_toolkit(index_tool_configs: dict[str, IndexToolConfig]) -> LlamaToolkit:
    """Create LangChain toolkit."""
    return LlamaToolkit(index_configs= list(index_tool_configs.values()))

def create_langchain_chat_agent(toolkit: LlamaToolkit) -> AgentExecutor:
    """Create LangChain chat agent."""
    memory = ConversationBufferMemory(memory_key="chat_history")
    llm=ChatOpenAI(temperature=0, model_name="gpt-4")
    return create_llama_chat_agent(
        toolkit,
        llm,
        memory=memory,
        verbose=True
    )


In [7]:
toolkit = create_langchain_toolkit(create_vector_index_tool_configs(vector_indexes))
agent = create_langchain_chat_agent(toolkit)

# Ask Questions For Testing

In [48]:
agent.run(input="Hi GPT, I'm Arthur, king of the britons.")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Hello Arthur, King of the Britons! How can I assist you today?

> Finished chain.


'Hello Arthur, King of the Britons! How can I assist you today?'

In [49]:
answer = agent.run(input="Give me a bullet point list of the three most important aspects that rijkswaterstaat considers when awarding a contract in dutch.")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Hallo Arthur, Koning van de Britten! Hier zijn de drie belangrijkste aspecten die Rijkswaterstaat in overweging neemt bij het toekennen van een contract:

1. Kwaliteit van de aanbieding: Rijkswaterstaat beoordeelt de technische bekwaamheid, professionaliteit en de voorgestelde methodologie van de aanbieder.
2. Prijs: De financiële aanbieding van de aanbieder wordt ook in overweging genomen. Rijkswaterstaat zoekt naar de beste waarde voor hun investering.
3. Duurzaamheid: Rijkswaterstaat hecht veel waarde aan duurzaamheid en milieu-impact. Aanbieders die duurzame oplossingen en innovaties voorstellen, worden vaak voorgetrokken.

Let op: Dit is een algemene lijst en de specifieke criteria kunnen variëren afhankelijk van het specifieke project of contract.

> Finished chain.


In [50]:
# Display markdown answer
from IPython.display import Markdown
Markdown(answer)

Hallo Arthur, Koning van de Britten! Hier zijn de drie belangrijkste aspecten die Rijkswaterstaat in overweging neemt bij het toekennen van een contract:

1. Kwaliteit van de aanbieding: Rijkswaterstaat beoordeelt de technische bekwaamheid, professionaliteit en de voorgestelde methodologie van de aanbieder.
2. Prijs: De financiële aanbieding van de aanbieder wordt ook in overweging genomen. Rijkswaterstaat zoekt naar de beste waarde voor hun investering.
3. Duurzaamheid: Rijkswaterstaat hecht veel waarde aan duurzaamheid en milieu-impact. Aanbieders die duurzame oplossingen en innovaties voorstellen, worden vaak voorgetrokken.

Let op: Dit is een algemene lijst en de specifieke criteria kunnen variëren afhankelijk van het specifieke project of contract.

In [51]:
answer = agent.run(input="Vat voor mij de aanbestedingsleidraad voor rijkswaterstaat opdrachten samen in een paar punten.")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Vector Index Aanbestedingsleidraad_BOC_ZD
Action Input: Vat voor mij de aanbestedingsleidraad voor rijkswaterstaat opdrachten samen in een paar punten.
Observation: 
1. De aanbestedingsprocedure is gebaseerd op de aanbestedingsleidraad, de nota's van inlichtingen, de processen-verbaal en de contractdocumenten.
2. Door het doen van een verzoek tot deelneming (aanmelding) en/of een inschrijving verklaart een ondernemer zich onvoorwaardelijk akkoord met de in deze aanbestedingsleidraad beschreven aanbestedingsprocedure.
3. De aanbestedingsprocedure betreft een opdracht voor werken met zaaknummer 31179058 voor het BasisOnderhoudsContract ZD A Wegen.
4. In afwijking op het ARW 2016 is bepaald dat het ontbreken van een inschrijvingsbiljet een gebrek is dat voor herstel vatbaar is.
5. Rijkswaterstaat hanteert de Gedragscode ‘Publiek Opdrachtgeverschap’.


> Finished chain.


In [53]:
Markdown(answer)


1. De aanbestedingsprocedure is gebaseerd op de aanbestedingsleidraad, de nota's van inlichtingen, de processen-verbaal en de contractdocumenten.
2. Door het doen van een verzoek tot deelneming (aanmelding) en/of een inschrijving verklaart een ondernemer zich onvoorwaardelijk akkoord met de in deze aanbestedingsleidraad beschreven aanbestedingsprocedure.
3. De aanbestedingsprocedure betreft een opdracht voor werken met zaaknummer 31179058 voor het BasisOnderhoudsContract ZD A Wegen.
4. In afwijking op het ARW 2016 is bepaald dat het ontbreken van een inschrijvingsbiljet een gebrek is dat voor herstel vatbaar is.
5. Rijkswaterstaat hanteert de Gedragscode ‘Publiek Opdrachtgeverschap’.

In [54]:
answer = agent.run(input="Welke documenten moeten bij aanmelding verstrek worden volgens het aanbestedingsleidraad document voor rijkswaterstaat?")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Vector Index Aanbestedingsleidraad_BOC_ZD
Action Input: Welke documenten moeten bij aanmelding verstrekt worden volgens het aanbestedingsleidraad document voor rijkswaterstaat?
Observation: 
Bij aanmelding moeten volgens het aanbestedingsleidraad document voor Rijkswaterstaat de volgende documenten verstrekt worden: Aanmeldingsformulier (Bijlage A), Uniform Europees Aanbestedingsdocument (Bijlage B), Aanvullende eigen verklaring (Bijlage C), Model indieningsformulier voor documenten van andere natuurlijke of rechtspersonen op wie de gegadigde/inschrijver zich beroept die met een handgeschreven handtekening worden ingediend (Bijlage D), Gegevens omtrent technische bekwaamheid (Bijlage E), Inschrijvingsbiljet (Bijlage F), Model K verklaring bestuurder omtrent rechtmatigheid inschrijving (Bijlage G), Verklaring inzake de verplichtingen op het gebied van milieu-, sociaal en arbeidsrecht (Bijlage I), Format Concept sta

In [55]:
answer = agent.run(input="Welke documenten zijn doorslaggevend voor de selectie van rijkswaterstaat? Leg uit waarom.")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: De selectie van Rijkswaterstaat is gebaseerd op een reeks documenten die door de aanbieder worden verstrekt. Deze kunnen variëren afhankelijk van het specifieke project of contract, maar over het algemeen zijn de volgende documenten doorslaggevend:

1. Aanmeldingsformulier en Uniform Europees Aanbestedingsdocument: Deze documenten geven Rijkswaterstaat een eerste indruk van de aanbieder en zijn capaciteiten. Ze bevatten basisinformatie over de aanbieder, zoals de bedrijfsnaam, het adres, de contactgegevens, de bedrijfsstructuur en de financiële situatie.

2. Technische bekwaamheid: Dit document geeft Rijkswaterstaat inzicht in de technische capaciteiten van de aanbieder. Het kan informatie bevatten over eerdere projecten, technische vaardigheden en ervaring, en andere relevante technische informatie.

3. Inschrijvingsbiljet: Dit document bevat de financiële aanbieding van de aanbieder. Het is een belangrijk document om

In [59]:
answer = agent.run(input="Gegeven dit aanbestdingsleidraad document, hoe zou een bedrijf zichzelf zo aantrekkelijk mogelijk kunnen maken voor de selectie? Geef kort en bondig antwoord.")



> Entering new  chain...
AI: Een bedrijf kan zichzelf aantrekkelijk maken voor de selectie door:

1. Duidelijke en volledige documentatie: Zorg ervoor dat alle vereiste documenten volledig en nauwkeurig zijn ingevuld. Dit toont professionaliteit en aandacht voor detail.

2. Bewijs van technische bekwaamheid: Toon eerdere projecten of ervaringen die aantonen dat het bedrijf de technische vaardigheden en capaciteiten heeft om het project succesvol uit te voeren.

3. Concurrerende prijsstelling: Bied een concurrerende prijs die waarde biedt voor Rijkswaterstaat. Dit toont aan dat het bedrijf de financiële stabiliteit en efficiëntie heeft om het project uit te voeren.

4. Duurzaamheid en maatschappelijk verantwoord ondernemen: Toon aan dat het bedrijf waarde hecht aan duurzaamheid en maatschappelijk verantwoord ondernemen. Dit kan worden aangetoond door middel van milieuvriendelijke praktijken, eerlijke arbeidspraktijken en betrokkenheid bij de gemeenschap.

5. Innovatie: Toon aan dat he

In [60]:
Markdown(answer)

Een bedrijf kan zichzelf aantrekkelijk maken voor de selectie door:

1. Duidelijke en volledige documentatie: Zorg ervoor dat alle vereiste documenten volledig en nauwkeurig zijn ingevuld. Dit toont professionaliteit en aandacht voor detail.

2. Bewijs van technische bekwaamheid: Toon eerdere projecten of ervaringen die aantonen dat het bedrijf de technische vaardigheden en capaciteiten heeft om het project succesvol uit te voeren.

3. Concurrerende prijsstelling: Bied een concurrerende prijs die waarde biedt voor Rijkswaterstaat. Dit toont aan dat het bedrijf de financiële stabiliteit en efficiëntie heeft om het project uit te voeren.

4. Duurzaamheid en maatschappelijk verantwoord ondernemen: Toon aan dat het bedrijf waarde hecht aan duurzaamheid en maatschappelijk verantwoord ondernemen. Dit kan worden aangetoond door middel van milieuvriendelijke praktijken, eerlijke arbeidspraktijken en betrokkenheid bij de gemeenschap.

5. Innovatie: Toon aan dat het bedrijf innovatief is en in staat is om nieuwe en creatieve oplossingen te bedenken. Dit kan Rijkswaterstaat helpen om hun doelen op een efficiëntere en effectievere manier te bereiken.